In [1]:
import pandas as pd
import numpy as np

import re

In [2]:
# Data input
demand = pd.read_excel('Demand_data1.xlsx')
#chargepoints = pd.read_excel('Charging_points.xlsx')
#potentialcharge = pd.read_excel('Potential_charging_points.xlsx')
#interest = pd.read_excel('Interest _points.xlsx')

In [3]:
# Defining the index sets
#number_of_periods = 4
number_of_grids = 434
type_of_chargers = 3

number_of_chargers = 78
number_of_pchargers = 325

#Periods = range(number_of_years)
Grids = range(number_of_grids)
Chargers = range(type_of_chargers)

charger_names = ['Slow', 'Fast', 'Rapid']
chargerpower = [2750, 4600, 40250]
chargerprice = [2500, 4000, 8000] ## for slow, fast, rapid chargers
concost = 100000 ## infrastructure construction cost


# Creating various variables

# demand matrix for each grid -- year 4 demand
targetdemand = demand['Demand_3']
targetdemand = targetdemand.to_numpy()

# maximum number of charging stations in each charging point 
maxchargestation = [20 for i in range(number_of_chargers+number_of_pchargers)] ## 20 as default bound


#Currentchargers = chargepoints[['Type','grid_number']]
# Current chargers availability:
currentchargers = demand[['Number_of Slow_Charging_Points','Number_of_Fast_Charging_Points','Number_of Rapid_Charging_Points']]
currentchargers = currentchargers.to_numpy()

# Potential chargers
pochargers = demand[['Ref', 'Number_of_Potential_Locations']]
pochargers = pochargers.to_numpy()

In [4]:
## Another way to input 'num_of_share'
## ‘num_of_share’: matrix of the number of neighbors plus 1 (itself) [434*1]
## 3 cases: 4 (vertices), 6 (sided except vertices), 9 (others)
# num_of_share = np.full(number_of_grids,9)
# ## some special points in the sides and vertices
# ## vertices: G1,14,421,434 (-1 for index) -- vertices only have 3 neighbours
# for i in {1,14,421,434}:
#     num_of_share[i-1] = 4
# for j in range(29):
#     num_of_share[1+14*(j+1)-1] = 6
#     num_of_share[14+14*(j+1)-1] = 6
# num_of_share[1:13]=6
# num_of_share[421:433]=6


In [5]:
## 'neighbours': matrix of neighbors and itself [343*9, 0 will be filled if num_of_share <9]
## each grid and its neighbours can support charging
neighbors = demand['NEIGHBORS']

neighbours = np.zeros((number_of_grids,9))
for g in Grids:
    nums = neighbors.iloc[g]
    num = re.findall(r'\d+', nums)
    num = list(map(int, num))
    num = np.append(arr = np.array([g+1]), values = num, axis= 0) ## add itself to the start
    #num.append(g+1) ## add itself
    while len(num)<9: ## fill in zeroes for grids with less than 9 neighbors
        #num.append(0)
        num = np.append( arr = np.array([0]), values = num, axis=0)
        
    neighbours[g,]=num
    
neighbours = neighbours.astype(int) ## convert to an integer array

In [6]:
for g in Grids:
    for i in range(9):
            ne = neighbours[g,i]
            if ne >0:
            ## remove those grids with no demand (based on Year 4 data)
                if (targetdemand[ne-1]==0) : 
                    neighbours[ne-1,][neighbours[ne-1,]== (g+1)] = 0 
                    neighbours[g,i] = 0 

In [7]:
num_of_share = (neighbours != 0).sum(1)

In [8]:
## 'cpower': the total charging power that can be provided by each grid (only consider current charging points)
cpower = np.zeros(number_of_grids)
for g in Grids:
    for c in Chargers:
        cpower[g] = cpower[g] + chargerpower[c]*currentchargers[g,c]

In [9]:
## 'tpower': total power that will be allocated to each gird
tpower = np.zeros(number_of_grids)

for g in Grids: ## each grid
    for ne in neighbours[g,]: ## each neighbour and itself
        if ne>1:
            tpower[g] = tpower[g] + cpower[ne-1]/num_of_share[ne-1]


In [10]:
## 'diff': the difference between demand and supply for each grid
diff = np.zeros(number_of_grids)
for g in Grids:
    diff[g] = diff[g] + targetdemand[g] - tpower[g]

## diff[74:78]

In [11]:
## find the grid with  
max( diff[g] for g in Grids ) ## G199

## v2: 72020.92700438993
## v3: 70466.49786052073
## v4: 68677.60897163185

71536.63674940962

In [12]:
## calculate the total unmet demand
unmet = 0

for g in Grids:
    if diff[g] > 0:
        unmet = unmet + diff[g]

unmet

## Using the assumption of one grid evenly sharing its power to all its neighbours
## original unmet demand: 732791.6610353214

## Adding the constraint of not sharing power to grids without demand
## updated unmet demand: 653338.3958766445

653338.3958766445

In [13]:
for g in Grids:
    if (diff[g] > 10000) & (diff[g] < 20000) :
        print(g+1, diff[g])

119 10126.838849934848
148 14431.673427852998
149 10714.232983138912
160 17582.302416328734
161 10724.565650181456
185 13427.481649888614
186 11486.046538966257
198 11314.9174817399


In [14]:
## expenditure
budget = 0

for g in Grids:
    for c in Chargers:
        budget += chargerprice[c]*currentchargers[g,c]
    

    
budget = budget + number_of_chargers*concost
    
print(budget)

8093500


In [15]:
## check the grids with unmet demand
#i = 0
#for g in Grids:
#    if diff[g] > 0:
#        i = i +1
#        print(g, diff[g])
        
#print(i)